In [1]:
from py5canvas import *

# Classes and dicts

Let's start with Python dictionaries (the `dict` type). We have seen these already briefly at the beginning of the module, but will repeat here. 
Dictionaries have a syntax very similar to general JavaScript "objects", and to the JSON (JavaScript Object Notation) file format. They allow to store "key"/"value" pairs, where the key is usually (but not necessarily) a string. E.g. 

In [2]:
person = {"name": "Jason",
          "surname": "Json",
          "age":99}
person

{'name': 'Jason', 'surname': 'Json', 'age': 99}

While in JavaScript this would be an "object" and we would access its entries with the dot notation (e.g. `person.name`), the Python syntax requires to access using an array-like notation with the key between square brackets, e.g.

In [3]:
person['name']

'Jason'

Py5canvas uses and comes with an external library called "easydict", which can be used to convert dictionaries to objects that enable the dot notation. Usually we import EasyDict as follows

In [5]:
from easydict import EasyDict as edict

Meaning that we import the `EasyDict` object from the module `easydict` but refer to it as `edict` (to save typing energy for more important things). So we can do 

In [6]:
p = edict(person)
print(p.name)

Jason


We will use this syntax to create a UI for our interactive sketches. We can do so by implementing a `parameters()` function that returns a dictionary:

In [ ]:
def parameters():
    params = {'Amount': (0.0, {'min':0.01, 'max':1.0}),
              'Background color': ([255, 0, 128], {'type':'color'}), 
               'A text field': 'Hello'}
    return params

With this code in your sketch, you will be able to modify these parameters using a global `params` easy dict, where the entries will be the name you gave a parameter, all lower case and with spaces replaced by underscores (`_`). So for example the "Background color" property will be accessible with `params.background_color`.

##  Classes
Python is an object oriented language, meaning that you can define "things" (i.e. objects) by creating classes. A class is like a blueprint or a template for creating objects. Imagine you’re building a car. A class would describe what a car is—how many wheels it has, what color it can be, and how it can move. But it’s not an instantiation of an actual car yet, it’s just the design for what an actual car could be.

A class typically includes:

**Attributes**:
These are like variables specific to the class or the objects created from the class.
They describe the "properties" of an object. For example, a car’s color, brand, and number of wheels.
Objects can have their own unique values for these attributes.

**Methods**:
These are like functions specific to the class and the objects created from it.
They define the "behaviors" of the object—what the object can do or how it can interact with the world. For example, a car might have a `drive()` method or a `honk()` method.

**Constructor**:
This is a special method in Python (called `__init__`) that defines how to create an object (or instance) from the class.
It allows you to set up attributes when an object is created.

Similarly to JavaScript, defininig a class requires having access to a variable that internally refers to the instance of the class that has been created. In Python this variable is defined with a special attribute called `self` (as opposed to `this` used in JS).

Differently from most Python code using lower-case and "snake case" (`snake_case`), the standard is to define classes in Python using a capital for the first letter of its name and camel case. E.g, we could define a class `FantasticCar`. A reader of the code usually knows that because of the capital, `FantasticCar` is a class name:


In [8]:
# Define a class (blueprint)
class FantasticCar:
    def __init__(self, color, brand):  # Constructor to set up the object
        self.color = color  # Attribute: color of the car
        self.brand = brand  # Attribute: brand of the car
    
    def drive(self):  # Method: behavior of the car
        print("The ", self.color, " ", self.brand, " car is driving!")
    
    def honk(self):  # Method: another behavior
        print("Honk! Honk!")

# Create an object (instance) from the class
my_car = FantasticCar(color="Red", brand="Toyota")  # Instance of Car class
print(my_car.color)  # Accessing an attribute: Output => Red
print(my_car.brand)  # Accessing an attribute: Output => Toyota

# Use methods
my_car.drive()  # Output => The Red Toyota car is driving!
my_car.honk()   # Output => Honk! Honk!

# Create another object (instance)
another_car = FantasticCar(color="Blue", brand="Fiat")
another_car.drive()  # Output => The Blue Honda car is driving!

Red
Toyota
The  Red   Toyota  car is driving!
Honk! Honk!
The  Blue   Fiat  car is driving!


Note that each "method" of the class has a first parameter called `self`. This is the syntax used by Python to inform the function (method) about the specific instance of the class that has been created, giving the method access to the parameters of the instance (e.g. here `self.brand` is the brand).

How is this useful? Prototypical use of a classes is to abstract and organize data for objects in a game or interactive environment, e.g. a player, an enemy a particle system, a particle etc. An example of a class we have used is `VideoInput` in Py5Canvas. It contains the functionalities that you need to play movies or get video from the camera. To get access to these functionalities you "instantiate" a `VideoInput` object using a constructor that specifies its properties (e.g. the size of a frame). As a practical example for this session we will see a particle, which can have a position a velocity an acceleration and a "lifetime".

This could be something like this:

In [10]:
class Particle:
    def __init__(self, lifetime=3.0):
        self.lifetime = lifetime
        self.life = self.lifetime
        self.reset()

    def reset(self):
        self.pos = np.array(center)
        self.vel = np.zeros(2)
        angle = -PI/2 + np.random.uniform(-1, 1)*0.2
        self.acc = np.array([np.cos(angle), np.sin(angle)])*400*np.random.uniform(0.5, 1.0)
        self.life = self.lifetime
    
    def update(self, dt, force=np.zeros(2)):
        self.acc += force
        self.acc += Vector(0, 9.8) # Gravity
        self.vel += self.acc*dt 
        self.pos += self.vel*dt
        self.life -= dt
        if self.life <= 0:
            self.reset()

    def draw(self):
        fill(255, 255*(self.life/self.lifetime))
        circle(self.pos, 2)


Here we give a particle a `lifetime` in seconds and a member `life` that decreases each time the `update` method is called. Once `life` reaches zero, the particle is re-initialized.

Here a particle is initialized with the `reset` method when it is constructed (in the `__init__` specialized constructor method) or when `life` reaches zero. Note that in `reset` we set the initial position to the built in vector `center`, which corresponds to the center of the canvas. You can set this to any position you like, e.g (`mouse_pos`).

The `update` method takes care of the motion of a particle and relies on a parameter `dt` that gives the time step for a tick of the simulation. In practice, the smaller this value, the more accurate the simulation would be. But we can use 1/60, taking into account the frame rate of a sketch. The procedure then consists of adding forces to the acceleration of the particle (e.g. gravity), adding the acceleration to the velocity of the particle, and finally adding the velocity to the position, thus resulting in the motion. We multiply by `dt` since velocity and accelerations are rates of change that depend on time, e.g. velocity is "change in position per second" and acceleration is "change in velocity per second".





